# Predicción calidad del vino

#### Javier Andrés Téllez Ortiz - 201617861

In [1]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


Incialmente, se descarga el dataset y se descomprime

In [1]:
import wget
from zipfile import ZipFile

##Se descarga el archivo del repositorio 
file = wget.download("http://www3.dsi.uminho.pt/pcortez/wine/winequality.zip")

##Se abre el archivo y se descomprime
zpFile = ZipFile(file)
zpFile.extractall()
zpFile.close()

100% [..............................................................................] 96005 / 96005

Se abre el archivo .csv y se analiza el tipo de datos obtenidos

In [2]:
import pandas as pd

##Se abre el archivo con más datos disponibles
dataframe = pd.read_csv("winequality/winequality-white.csv", delimiter = ";")

##Se muestra la composición de los datos
dataframe.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


Todas las características son numéricas y se tienen 11 diferentes variables para cada muestra. Se eliminan las muestras con algún dato faltante:

In [7]:
print("Cantidad de datos antes de eliminar faltantes: %d" % dataframe.shape[0])

##Se eliminan los datos con alguna caracteristica faltante
dataframe.dropna()
print("Cantidad de datos disponibles: %d" % dataframe.shape[0])


Cantidad de datos antes de eliminar faltantes: 4898
Cantidad de datos disponibles: 4898


Se generan los histogramas correspondientes a cada característica. Se evidencia que la distribución de algunas carácteristicas no se aproxima adecuadamente a una distribución normal.

In [4]:
##Se obtienen los nombres de los parámetros del modelo del dataset
features = dataframe.columns.tolist()

##Se obtienen los valores y se dividen entre datos y objetivos
values = dataframe.values

y = values[:,-1]
X = values[:,0:-1]

##Se genera un histograma para cada caracteristica
##figure = dataframe.hist(figsize=(15,15), bins=50)
X

array([[ 7.  ,  0.27,  0.36, ...,  3.  ,  0.45,  8.8 ],
       [ 6.3 ,  0.3 ,  0.34, ...,  3.3 ,  0.49,  9.5 ],
       [ 8.1 ,  0.28,  0.4 , ...,  3.26,  0.44, 10.1 ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  2.99,  0.46,  9.4 ],
       [ 5.5 ,  0.29,  0.3 , ...,  3.34,  0.38, 12.8 ],
       [ 6.  ,  0.21,  0.38, ...,  3.26,  0.32, 11.8 ]])

Con el objetivo de evidenciar el impacto que tiene sobre el modelo la estandarización de los datos, se hará el proceso tanto con los datos "crudos" y los datos preprocesados

In [77]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

##Se estandarizan los datos provenientes del archivo
normalized_values = scaler.fit(values).transform(values)

##Se dividen entre datos y objetivos
normalized_y = normalized_values[:,-1]
normalized_X = normalized_values[:,0:-1]

Se dividen los datos entre conjuntos de prueba y de entrenamiento

In [78]:
from sklearn.model_selection import train_test_split

##Se dividen aleatoriamente los datos entre prueba y entrenamiento para ambos casos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4083, random_state = 7861)

X_train_std, X_test_std, y_train_std, y_test_std = train_test_split(normalized_X, normalized_y,
                                                                    test_size = 0.4083, random_state = 7861) 

print("Tamaño conjunto de prueba: %d" % y_test.size)

Tamaño conjunto de prueba: 2000


Se procede a entrenar el modelo con los diferentes tamaños del conjunto con este fin:

In [137]:
from sklearn.linear_model import LinearRegression

#Se entrena el modelo con 100 datos
regression_100 = LinearRegression().fit(X_train[0:101], y_train[0:101])
regression_100_std = LinearRegression().fit(X_train_std[0:101], y_train_std[0:101])

#Se entrena el modelo con 1000 datos
regression_1000 = LinearRegression().fit(X_train[0:1001], y_train[0:1001])
regression_1000_std = LinearRegression().fit(X_train_std[0:1001], y_train_std[0:1001])

#Se entrena el modelo con 2898 datos
regression = LinearRegression().fit(X_train, y_train)
regression_std = LinearRegression().fit(X_train_std, y_train_std)

Se imprimen los coeficientes para las diferentes características de las muestras. Se evidencia la diferencia que existe entre los coeficientes para los modelos entrenados con preprocesamiento y sin preprocesamiento

In [138]:
regression_model = pd.DataFrame({"Característica" : features[0:-1],
                                "100" : regression_100.coef_,
                                "1000" : regression_1000.coef_ ,
                                "2898" : regression.coef_ ,
                                "100 (std)" : regression_100_std.coef_,
                                "1000 (std)" : regression_1000_std.coef_ ,
                                "2898 (std)" : regression_std.coef_ 
                                })

regression_model

,Característica,100,1000,2898,100 (std),1000 (std),2898 (std)
0,fixed acidity,0.313415,0.085306,0.035540,0.298633,0.081283,0.033863
1,volatile acidity,-2.180485,-1.949464,-1.982060,-0.248161,-0.221869,-0.225578
2,citric acid,-0.640552,0.241144,0.025404,-0.087529,0.032952,0.003471
3,residual sugar,0.103425,0.094544,0.073473,0.592315,0.541452,0.420781
4,chlorides,-0.352402,-0.459678,-0.570381,-0.008693,-0.011340,-0.014071
5,free sulfur dioxide,0.000093,0.004553,0.004613,0.001787,0.087438,0.088576
6,total sulfur dioxide,-0.004003,-0.000753,-0.000434,-0.192101,-0.036126,-0.020836
7,density,-186.515654,-188.354342,-121.624636,-0.629886,-0.636095,-0.410741
8,pH,1.219749,0.829820,0.574898,0.207966,0.141484,0.098020
9,sulphates,1.019677,0.678611,0.517718,0.131398,0.087448,0.066715


Se prueba cada uno de los modelos obtenidos con el conjunto de prueba correspondiente

In [140]:
error_medio = []
error_cuadrático_medio = []
r2_score = []

models = [regression_100, regression_1000, regression, 
          regression_100_std, regression_1000_std, regression_std]

test_data = [X_test, X_test, X_test, X_test_std, X_test_std, X_test_std]
test_target = [y_test, y_test, y_test, y_test_std, y_test_std, y_test_std]

for model,data,target in zip(models, test_data, test_target):
    
    y_hat = model.predict(data)
    error_medio.append( np.mean( np.absolute(y_hat - target)))
    error_cuadrático_medio.append( np.mean((y_hat - target) ** 2))
    r2_score.append(model.score(data, target))
    
model_results = pd.DataFrame({
    "Número de datos" : ["100", "1000", "2898", "100(std)", "1000(std)", "2898(std)"],
    "Error medio" : error_medio,
    "Error cuadrático medio" : error_cuadrático_medio,
    "R2" : r2_score    
})

model_results

,Número de datos,Error medio,Error cuadrático medio,R2
0,100,0.625835,0.648353,0.182089
1,1000,0.595224,0.590997,0.254445
2,2898,0.596128,0.592829,0.252133
3,100(std),0.706721,0.826774,0.182089
4,1000(std),0.672153,0.753635,0.254445
5,2898(std),0.673175,0.755971,0.252133


## Conclusiones
- Es posible afirmar que, a partir de los puntajes R2 obtenidos, un modelo lineal no es el adecuado para el objetivo planteado.
- Por otro lado, se evidencio que la normalización de los datos no tuvo ningún efectivo positivo en el desempeño del modelo.
- Debido a que las calidades del vino son variables discretas ordinales, se puede afirmar que una regresión lineal no captura adecuadamente la relación entre las variables y la predicción a realzar.

# Bilbiografia
- P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.